Find all TCP and UDP streams sequence numbers from pcap

In [ ]:
%%bash 
tshark -r Infiltration_Dropbox_P2_All.pcap -Y "tcp || udp" -T fields -e tcp.stream -e udp.stream | sort | uniq > Infiltration_Dropbox_P2_all_streams.txt

Take 1% of the total TCP and UDP streams sequence numbers from pcap

In [ ]:
%%bash 
total_streams=$(wc -l < Infiltration_Dropbox_P2_all_streams.txt)
select_count=$((total_streams * 10 / 100))  # Adjust the percentage here
shuf Infiltration_Dropbox_P2_all_streams.txt | head -n $select_count > Infiltration_Dropbox_P2_selected_streams.txt

Extract the selected TCP and UDP streams from pcap

In [ ]:
%%bash
output_file="Infiltration_Dropbox_P2_All.pcap"
> $output_file

# Determine chunk size, e.g., 100 streams per chunk
chunk_size=100
total_lines=$(cat Infiltration_Dropbox_P2_selected_streams.txt | wc -l)
num_chunks=$((total_lines / chunk_size + (total_lines % chunk_size > 0)))

for ((i = 1; i <= num_chunks; i++)); do
    start_line=$(( (i - 1) * chunk_size + 1 ))
    end_line=$(( i * chunk_size ))
    
    filter=""
    for stream in $(sed -n "${start_line},${end_line}p" Infiltration_Dropbox_P2_selected_streams.txt); do
        filter="${filter:+$filter || }tcp.stream==$stream || udp.stream==$stream"
    done
    
    tshark -r Infiltration_Dropbox_P2_All.pcap -Y "$filter" -w "Infiltration_Dropbox_P2_chunk_${i}.pcap"
done

# Merge all chunks
mergecap -w $output_file Infiltration_Dropbox_P2_chunk_*.pcap

# Cleanup
rm Infiltration_Dropbox_P2_chunk_*.pcap